นาย ปพนธ์ ชุณหคล้าย 6210503691

In [ ]:
from __future__ import unicode_literals

import json
from time import sleep
from bs4 import BeautifulSoup

In [ ]:

from kafka import KafkaConsumer, KafkaProducer

import requests
# -*- coding: utf-8 -*-


In [ ]:
def fetch_raw(recipe_url):
    html = None
    print('Processing..{}'.format(recipe_url))
    try:
        r = requests.get(recipe_url, headers=headers)
        if r.status_code == 200:
            html = r.text
    except Exception as ex:
        print('Exception while accessing raw html')
        print(str(ex))
    finally:
        return html.strip()


def get_recipes():
    recipies = []
    # salad_url = 'https://www.allrecipes.com/recipes/156/bread/'
    url = 'https://gamechangersmovie.com/food/recipes/?fwp_recipe_categories=lunches-dinners&fwp_paged=2s&#39;'
    print('Accessing list')

    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            html = r.text
            #soup = BeautifulSoup(html, 'lxml')
            soup = BeautifulSoup(html, 'html.parser')
            #soup = BeautifulSoup(r.content, features="html")
            links = soup.select('.tag-title')
             
            idx = 0
            print('total',len(links))
            for link in links:

                sleep(2)
                #print(link)
                recipe = fetch_raw(link['href'])
                recipies.append(recipe)
                
                #print(link['href'])
                idx += 1
                if idx > 11:
                    break
    except Exception as ex:
        print('Exception in get_recipes')
        print(str(ex))
    finally:
        return recipies

In [ ]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        #print(key)
        #print(value)
        key_bytes =  bytes(key, encoding='utf-8')   #.encode('utf-8')
        value_bytes = bytes(value, encoding='utf-8') #encode('utf-8')
        
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))


def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [ ]:
if __name__ == '__main__':
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
        'Pragma': 'no-cache'
    }
    all_recipes = get_recipes()
    if len(all_recipes) > 0:
        kafka_producer = connect_kafka_producer()
        for recipe in all_recipes:
             
            publish_message(kafka_producer, 'raw_recipes', 'raw', recipe.strip())
        if kafka_producer is not None:
            kafka_producer.close()

Accessing list
total 12
Processing..https://gamechangersmovie.com/recipes/bbq-chickpea-and-carrot-sliders/
Processing..https://gamechangersmovie.com/recipes/southwest-stew/
Processing..https://gamechangersmovie.com/recipes/the-whole-bowl-pan-roasted-tofu-broccoli-bites-and-wilted-spinach/
Processing..https://gamechangersmovie.com/recipes/tempeh-cauliflower-potato-curry/
Processing..https://gamechangersmovie.com/recipes/mac-n-cheese/
Processing..https://gamechangersmovie.com/recipes/black-bean-and-sweet-potato-quesadillas/
Processing..https://gamechangersmovie.com/recipes/quick-black-bean-tostadas/
Processing..https://gamechangersmovie.com/recipes/hearty-lentil-stew/
Processing..https://gamechangersmovie.com/recipes/tu-no-casserole/
Processing..https://gamechangersmovie.com/recipes/margherita-pizza/
Processing..https://gamechangersmovie.com/recipes/ramen/
Processing..https://gamechangersmovie.com/recipes/quinoa-bowl-with-roasted-sweet-taters-and-black-beans/
Message published successful